---
title: "End-to-End Benchmarking"
---

## Processing benchmark results

### Import dependencies

The CarbonPlan team put together some utilities for parsing, processing, and visualizing the benchmarking results in [carbonplan_benchmarks](https://github.com/carbonplan/benchmark-maps). We'll use those utilities along with the [Holoviz](https://holoviz.org/) HoloViz suite of tools for visualization and [Pandas](https://pandas.pydata.org/) as the underlying analysis tool.

In [1]:
import carbonplan_benchmarks.analysis as cba
import hvplot
import pandas as pd

pd.options.plotting.backend = "holoviews"

### Load benchmark results

First, define the paths to the baseline images that the tests will be compared against and paths to the metadata files associated with each benchmarking run.

In [2]:
baseline_fp = "s3://carbonplan-benchmarks/benchmark-data/baselines.json"
metadata_base_fp = "s3://carbonplan-benchmarks/benchmark-data"
metadata_files = [
    "data-2023-08-04T01-14-24.json",
    "data-2023-08-04T01-15-30.json",
    "data-2023-08-04T01-16-27.json",
    "data-2023-08-04T01-17-25.json",
    "data-2023-08-04T01-18-37.json",
    "data-2023-08-04T01-19-47.json",
    "data-2023-08-04T01-21-02.json",
    "data-2023-08-04T01-22-08.json",
]

Now, use the utilities from `carbonplan_benchmarks` to load the metadata and baseline images into DataFrames, process those results, and create a summary DataFrame for all runs.

In [3]:
snapshots = cba.load_snapshots(snapshot_path=baseline_fp)
summary_dfs = []
for file in metadata_files:
    fp = f"{metadata_base_fp}/{file}"
    metadata, trace_events = cba.load_data(metadata_path=fp, run=0)
    data = cba.process_run(
        metadata=metadata, trace_events=trace_events, snapshots=snapshots
    )
    summary_dfs.append(cba.create_summary(metadata=metadata, data=data))
summary = pd.concat(summary_dfs)

In [4]:
summary.head(n=8)

,action,browser_name,browser_version,playwright_python_version,provider,trace_path,url,zoom_level,approach,zarr_version,dataset,chunk_size,zoom,duration,fps,request_duration,request_percent
0,zoom_in,chromium,115.0.5790.75,1.36.0,unknown,s3://carbonplan-benchmarks/benchmark-data/2023...,https://prototype-maps.vercel.app/direct-clien...,4,direct-client,v2,1MB-chunks,1,0.0,1039.890,50.005289,962.687,92.575849
1,zoom_in,chromium,115.0.5790.75,1.36.0,unknown,s3://carbonplan-benchmarks/benchmark-data/2023...,https://prototype-maps.vercel.app/direct-clien...,4,direct-client,v2,1MB-chunks,1,1.0,1076.482,54.808162,738.299,68.584426
2,zoom_in,chromium,115.0.5790.75,1.36.0,unknown,s3://carbonplan-benchmarks/benchmark-data/2023...,https://prototype-maps.vercel.app/direct-clien...,4,direct-client,v2,1MB-chunks,1,2.0,702.452,59.790562,361.903,51.519962
3,zoom_in,chromium,115.0.5790.75,1.36.0,unknown,s3://carbonplan-benchmarks/benchmark-data/2023...,https://prototype-maps.vercel.app/direct-clien...,4,direct-client,v2,1MB-chunks,1,3.0,981.422,39.738257,646.134,65.836511
4,zoom_in,chromium,115.0.5790.75,1.36.0,unknown,s3://carbonplan-benchmarks/benchmark-data/2023...,https://prototype-maps.vercel.app/direct-clien...,4,direct-client,v2,1MB-chunks,1,4.0,475.924,60.934099,0.000,0.000000
0,zoom_in,chromium,115.0.5790.75,1.36.0,unknown,s3://carbonplan-benchmarks/benchmark-data/2023...,https://prototype-maps.vercel.app/direct-clien...,4,direct-client,v2,5MB-chunks,5,0.0,1199.615,34.177632,1107.553,92.325704
1,zoom_in,chromium,115.0.5790.75,1.36.0,unknown,s3://carbonplan-benchmarks/benchmark-data/2023...,https://prototype-maps.vercel.app/direct-clien...,4,direct-client,v2,5MB-chunks,5,1.0,1362.769,53.567406,996.513,73.124132
2,zoom_in,chromium,115.0.5790.75,1.36.0,unknown,s3://carbonplan-benchmarks/benchmark-data/2023...,https://prototype-maps.vercel.app/direct-clien...,4,direct-client,v2,5MB-chunks,5,2.0,2902.872,25.147509,2558.564,88.139057


## Visualize results

First, let's see how the duration of each action changes as a function of the zoom level. An important piece of context is that the underlying dataset only has four pyramid levels, so zoom=4 does not need to fetch any new data.

In [5]:
summary.plot.scatter(x="zoom", y="duration", by="zarr_version")

:NdOverlay   [zarr_version]
   :Scatter   [zoom]   (duration)

Now, let's instead show the duration as a function of the chunk size.

In [6]:
summary.plot.scatter(x="chunk_size", y="duration", by="zarr_version")

:NdOverlay   [zarr_version]
   :Scatter   [chunk_size]   (duration)

Now, let's look at the request duration as a funciton of the chunk size.

In [7]:
summary.plot.scatter(x="chunk_size", y="request_duration", by="zarr_version")

:NdOverlay   [zarr_version]
   :Scatter   [chunk_size]   (request_duration)

Lastly, let's look at the fraction of time that's spent fetching data as a function of the chunk size.

In [8]:
summary.plot.scatter(x="chunk_size", y="request_percent", by="zarr_version").opts(
    ylim=(0, 100)
)

:NdOverlay   [zarr_version]
   :Scatter   [chunk_size]   (request_percent)